## Install Library

In [ ]:
%pip install "autogen-agentchat==0.4.0.dev13" "autogen-ext[openai,azure]==0.4.0.dev13"

## Load Azure Configuration

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Quick Start

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# Define an agent
weather_agent = AssistantAgent(
    name="weather_agent",
    description="This agent provides weather information.",
    model_client=az_model_client,
    tools=[get_weather],
)

# Define a team with a single agent and maximum auto-gen turns of 1.
agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

while True:
    # Get user input from the console.
    user_input = input("Enter a message (type 'exit' to leave): ")
    if user_input.strip().lower() == "exit":
        break
    # Run the team and stream messages to the console.
    stream = agent_team.run_stream(task=user_input)
    await Console(stream)

## Text Message
Create a text message, which accepts a string content and a string source

In [ ]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="What's the weather in NY", source="User")

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task=text_message)
await Console(stream)

## MultiModal Message

In [17]:
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image
from matplotlib import pyplot as plt
import cv2

raw_image = Image.open("empirestate.jpg")


# Convert the image from BGR to RGB for displaying with matplotlib,
# because OpenCV uses BGR by default and matplotlib expects RGB.
img_rgb = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)

# Display the image with matplotlib.
plt.imshow(img_rgb)
plt.axis("off")  # Turn off axis labels.
plt.show()

# Create an AGImage object from the raw image.
img = AGImage(raw_image)
multi_modal_message = MultiModalMessage(content=["Identify the city in this image and tell me the weather", img], source="User")

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task=multi_modal_message)
await Console(stream)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


# Agents have State

In [ ]:
text_message = TextMessage(content="What was the city again that I asked about the weather?", source="User")

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task=text_message)
await Console(stream)